In [1]:
import pandas as pd
from datetime import datetime as dt
from pytz import timezone, utc
import re

In [2]:
time_stamp = dt.now()

'Approval Tool Data Cut On {} at {}'.format(time_stamp.date(), time_stamp.strftime('%I %p'))

'Approval Tool Data Cut On 2022-12-01 at 10 AM'

In [3]:
template_dict = {'[NACF] Startup CAR Approval Template'         : 7360, 
                 '[ECCF MEXICO] Startup CAR Approval Template'  : 7394, 
                 'NACF IXD Initiative Approval Template'        : 39171, 
                 'NACF AR Sortable Initiative Approval Template': 39172, 
                 'NACF ==TSSL, Traditional Sortable, Apparel, Shoes, Make On Demand, Merch by Amazon buildings, Dangerous Goods, Reverse Logistics== Initiative Approval Template': 39174,
                 'NACF Non-Sort Initiative Approval Template'   : 39175,
                 'ATS Initiative Approval Template'             : 39176, 
                 'EU Real Estate CAR Approval loop + Supplemental CARs + Lease renewals + CRAs': 37508}

approver_dict = {'SP': 'Perego, Stefano', 
#                 'OG' : 'Urroz Garcia, Olatz',
                 'DH': 'Heimburger, Doug',
#                  'MM' : 'Midgley, Maureen',
                 'MN' : 'Nick, Melissa', 
#                  'TR' : 'Riley, Toni', 
                 'RW' : 'Williams (Finance), Russel'}

actor_action_dict = {'Approved':'APPROVER_APPROVED', 
                     'Approved with condition':'APPROVER_APPROVED_WITH_CONDITION',
                     'Pending':'APPROVER_PENDING',
                     'Pending in future':'APPROVER_PENDING_IN_FUTURE'}

DIVIDER = ' | '

CHECK_MARK = '\u2713'
PENDING_MARK = ' '
DEFAULT_MARK = 'NA'

BUILDING_TYPE_LIST = ['ARS','ARS450',
                      'TNS','TNS80',
                      'Kariba',
                      'Hazmat',
                      'Sort Center',
                      'IXD',
                      'Mixed Sort',
                      'SFC Retrofit',
                      'TSSL','IPC','3PL']

FULL_CAR_TYPE_LIST = ['Initial Full Startup',
                      'Final Full Startup',
                      'Startup Supplemental',
                      'Supplemental Startup',
                      'Construction',
                      'Full Real Estate',
                      'Lease Renewal',
                      'Reallocation']

SORT_ORDER_REGION = {'EU': 0, 'AP': 1, 'IN': 2, 'EC':3, 'NA':4, 'Check':5}

In [4]:
input_file_path = 'data/GCF Weekly CAR Input/AmazonApprovals_20221201.xlsx' # new approval file shared from Christina
new_approval_report_output_file_path = 'data/GCF Weekly CAR Output/Approval_Report_{date}.xlsx'.format(date=str(dt.now().date()))


last_edit_report = 'data/GCF Weekly CAR Input/Weekly_CAR_Report_GCP_2022-11-23.xlsx' # downloaded from GCP team's workdoc folder
new_report = new_approval_report_output_file_path

final_output_file_path = 'data/GCF Weekly CAR Output/Weekly_CAR_Report_GCP_{date}.xlsx'.format(date=str(dt.now().date()))

In [5]:
data = pd.read_excel(input_file_path)



data.rename(columns={'approval_id':'Approval Id', 
                     'title':'Title', 
                     'needed_on':'Approval Needed By', 
                     'decision':'Actor Action', 
                     'approver':'Actor', 
                     'status':'Status',
                     'template_id':'Template Id'}, inplace=True)

In [6]:
if 'Template Id' not in data.columns:
    data['Template Id'] = 0

if 'Role' in data.columns:
    df = data.loc[(data['Status']=='Pending') & (data['Role']=='APPROVER')].reset_index(drop=True)
else:
    df = data.loc[(data['Status']=='Pending') & (~data['Actor Action'].isna())].reset_index(drop=True)

df = df.loc[df['Actor Action'].isin(['Pending', 'Pending in future', 'Approved', 'Approved with condistion'])].reset_index(drop=True)

In [7]:
df.head()

,Approval Id,Title,Approval Needed By,Actor Action,Actor,Status,Template Id
0,13519512,NA | AMZL CAR Approval | 2023 | AMZL | Un-Moth...,12-02-2022 05:59:59,Pending,"Smith, Zach",Pending,0
1,13519512,NA | AMZL CAR Approval | 2023 | AMZL | Un-Moth...,12-02-2022 05:59:59,Approved,"Roth, Emily",Pending,0
2,13519512,NA | AMZL CAR Approval | 2023 | AMZL | Un-Moth...,12-02-2022 05:59:59,Pending,"Rogers, Stephen",Pending,0
3,13519512,NA | AMZL CAR Approval | 2023 | AMZL | Un-Moth...,12-02-2022 05:59:59,Approved,"Gregory, Ryan",Pending,0
4,13519512,NA | AMZL CAR Approval | 2023 | AMZL | Un-Moth...,12-02-2022 05:59:59,Pending in future,Capital-Planning,Pending,0


In [8]:
def concate_unique_value(df):
    unique_value = df.transform(lambda x : ','.join(set(x)))
    unique_value['count'] = len(df)
    
    return unique_value

def get_nbd(df_template):
    # Get unique need by date(s) for each Approval Id & Title
    df_nbd = df_template.groupby(['Template Id','Approval Id','Title'])[['Approval Needed By']].agg(lambda x : DIVIDER.join(set(x)))
    df_nbd.reset_index(inplace=True)
    print(df_nbd.head())
    
    return(df_nbd)

def pivot_actor_action(df_template):
    # Get unique need by date(s) for each Approval Id & Title
    # Get unique actor(s) for both 'Pending' and 'Pending in future' Actor Action
    df_actors = df_template.groupby(['Template Id','Approval Id','Title','Actor Action'])[['Actor']].agg(lambda x : DIVIDER.join(set(x)))
    df_actors.reset_index(inplace=True)
    df_actors = df_actors.pivot(index=['Template Id','Approval Id','Title'], columns='Actor Action', values='Actor')
    df_actors.reset_index(inplace=True)
    
    return(df_actors)

def get_approver_check_list(df_pivot):
    output = []

    for i in range(len(df_pivot)):
        pending = str(df_pivot['Pending'][i])
        pending_in_future = str(df_pivot['Pending in future'][i])
        approved = str(df_pivot['Approved'][i])
        approved_with_condition = str(df_pivot['Approved with condition'][i])


        pending_list = pending.split(DIVIDER) + pending_in_future.split(DIVIDER)
        approved_list = approved.split(DIVIDER) + approved_with_condition.split(DIVIDER)

        check_list = {}

        # create default check_list with all approvers as DEFAULT_MARK
        for approver in approver_dict.keys():
            check_list[approver] = DEFAULT_MARK

        # loop through the pending list to check the approvers
        for approver, full_name in approver_dict.items():
            if full_name in pending_list:
                check_list[approver] = PENDING_MARK
            elif full_name in approved_list:
                check_list[approver] = CHECK_MARK

        output.append(check_list)
    print('pd.DataFrame(output)')
    print(pd.DataFrame(output))
    return(pd.DataFrame(output))

def determine_queue(df_info):
    print(df_info)
    df_info['Queue'] = 'Pre-VP'

    SP_pending_in_future = [approver_dict['SP'] in str(i) for i in df_info['Pending']]
    DH_pending_in_future = [approver_dict['DH'] in str(i) for i in df_info['Pending']]
    #OG_pending_in_future = [approver_dict['OG'] in str(i) for i in df_info['Pending']]
    MN_pending_in_future = [approver_dict['MN'] in str(i) for i in df_info['Pending']]
    
    vp_queue = [a | d | m for a,d, m in zip(SP_pending_in_future, DH_pending_in_future, MN_pending_in_future)]
    
    
    df_info.loc[vp_queue, 'Queue'] = 'VP'
    
    return(df_info)

def preprocess_title(title):
    result = title.replace('&ndash;','-').replace('&amp;','&').replace('[','').replace(']','')
    
    return(result)

def add_hyper_link_to_approval(approval_id):
    return('=HYPERLINK("https://approval.amazon.com/Approval/Details/{approval_id}", {approval_id})'.format(approval_id=approval_id))

def utc_to_pst(date_time):
    return(date_time.astimezone(timezone('US/Pacific')))

def format_nbd(need_by_date):
    return(pd.to_datetime(need_by_date, format='%m-%d-%Y %H:%M:%S', utc=True).apply(utc_to_pst).dt.strftime('%d-%b, %Y'))

def parse_site_name(title):
    corner_case = ['ARS','TNS']
    
    site_name_list = []
    
    for s in re.findall('[A-Z][A-Z][A-Z][0-9]',title):
        if not any([c in s for c in corner_case]):
            site_name_list.append(s)
    
    return(', '.join(list(set(site_name_list))))

def parse_building_type(title):
    separater = ' '
    building_type = ''
    
    for b in BUILDING_TYPE_LIST:
        if b in title:
            building_type = building_type + b + separater
            
    if len(building_type) == 0:
        building_type = ''
    else:
        building_type = building_type[:-len(separater)]
    
    return(building_type)

def parse_full_car_type(title):
    separater = ' and '
    full_car_type = ''
    
    for b in FULL_CAR_TYPE_LIST:
        if b in title:
            full_car_type = full_car_type + b + separater
            
    if len(full_car_type) == 0:
        full_car_type = ''
    else:
        full_car_type = full_car_type[:-len(separater)]
    
    return(full_car_type)

def get_first_non_blank_position_after_pos(pos, title):
    if pos == -1:
        return(-1)
    
    for p in range(pos+1,len(title)):
        if title[p] != ' ':
            return(p)
    
    return(-1)

def get_first_non_blank_position_after_pos(pos, title):
    if pos == -1:
        return(-1)
    
    for p in range(pos+1,len(title)):
        if title[p] != ' ':
            return(p)
    
    return(-1)

def get_valid_numeric_format(string):
    numeric_string = re.sub('[^0-9,.]', '', string)
    
    last_comma_pos = numeric_string.rfind(',')

    # Determine if the numeric string patter is "XXXX,X"
    if last_comma_pos+1 == len(numeric_string):
        last_comma_pos_in_original_str = string.rfind(',')
        string = string[:last_comma_pos_in_original_str] + string[last_comma_pos_in_original_str+1:]
    
    if len(numeric_string) - (last_comma_pos+1) < 3:
        last_comma_pos_in_original_str = string.rfind(',')
        string = string[:last_comma_pos_in_original_str] + string[last_comma_pos_in_original_str:].replace(',','.')
    
    return(string)
    
def parse_current_request(title):
    # Determine Current Request
    dollar_sign_position = title.find('$')
    
    dollar_sign_position = get_first_non_blank_position_after_pos(dollar_sign_position, title)
    
    if dollar_sign_position != -1:
        current_request = title[dollar_sign_position:].split(' ')[0].replace(')','').replace('$','')
        
        current_request = get_valid_numeric_format(current_request)
        try:
            if 'MM' in current_request or 'M' in current_request:
                current_request_amount = round(float(current_request.replace('MM','').replace('M','')), 1)
            elif 'k' in current_request or 'K' in current_request:
                current_request_amount = round(float(current_request.lower().replace('k','')) / 1000, 1)
            else:
                count_dot = len(re.findall('\.', current_request))

                if count_dot <= 1:
                    current_request_amount = float(current_request.replace(',','')) / 1000000
                    
                    if current_request_amount < 10000 and ('MM' in title[dollar_sign_position:] or 'M' in title[dollar_sign_position:]):
                        current_request_amount = current_request_amount * 1000000
                    elif current_request_amount < 10000 and ('k' in title[dollar_sign_position:] or 'K' in title[dollar_sign_position:]):
                        current_request_amount = current_request_amount * 1000
                    
                    current_request_amount = round(current_request_amount, 1)
                    
                else:
                    current_request_amount = None
                    unrecongized = True
        except:
            print('Unable to recognize the current request: {}'.format(current_request))
            current_request_amount = None
            unrecongized = True
    else:
        current_request_amount = None
        unrecongized = True
    
    return(current_request_amount)

def parse_por_year(title, parse_pattern=' {} '):
    por_year = ''
    por_year_list = range(dt.now().year-2,dt.now().year+5)

    for y in por_year_list:
        if parse_pattern.format(y) in title:
            por_year = por_year + str(y) + ', '
        
    if len(por_year) == 0:
        if parse_pattern == ' {} ': # If the default parse pattern doesn't find the por year in title, try different patterns and re parse the title again
            parse_pattern_list = ['{} ','{}_']
            
            for p in parse_pattern_list:
                por_year = parse_por_year(title, parse_pattern=p)
                
                if por_year != '':
                    break
        else:
            por_year = ''
            unrecongized = True
    else:
        por_year = por_year[:-2]
    
    return(por_year)

def parse_region(title, template_id):
    region = ''
    
    search_first_alphbet = None
    search_first_alphbet = re.search('[A-Z]', title, re.I)
    if search_first_alphbet is not None:
        pos = search_first_alphbet.start()
        title = title[pos:]
    
    # Template Id based information extraction (WIP)
    if template_id == 7394:
        region = 'EC'
    elif template_id == 30143:
        region = 'NA'
    elif template_id == 35306:
        region = 'NA'
    elif template_id == 35476:
        region = 'NA'
    elif template_id == 36470:
        region = 'NA'
        building_type = 'IXD'
    elif template_id == 37155:
        region = 'NA'
    elif template_id == 37156:
        region = 'NA'
    elif template_id == 37158:
        region = 'NA'
    elif template_id == 37508:
        region = 'EU'
    elif template_id == 39802:
        region = 'NA'
    elif template_id == 39818:
        region = 'NA'
    elif template_id == 39820:
        region = 'NA'
    
    if region == '':
        region = title[:2]
    if region in ['JP', 'IN', 'SIN']:
        region = 'AP'
    if region in ['AU', 'MX', 'BR']:
        region = 'EC'
    if region not in SORT_ORDER_REGION.keys():
        region = 'Check'
    
    return(region)

def extract_info_from_title(title, template_id):
    region = ''
    por_year = ''
    car_type = ''
    building_type = ''
    current_request = ''
    unrecongized = False
    
    title = preprocess_title(title)
    
    # Determine CAR Type
    if 'startup' in title.lower() or ' SU ' in title:
        if 'real estate' in title.lower() or ' RE ' in title:
            car_type = 'RE & SU'
        else:
            car_type = 'SU'
    elif 'real estate' in title.lower() or ' RE ' in title:
        car_type = 'RE'
    elif 'construction' in title.lower() or ' CN ' in title:
        car_type = 'RE'
    elif 'initiative' in title.lower():
        car_type = 'Initiative'
    else:
        car_type = None
        unrecongized = True
    
    # Determine POR Year
    por_year = parse_por_year(title)
    
    # Determine Current Request
    current_request = parse_current_request(title)
    
    # Parse Site Nmae
    site = parse_site_name(title)
    
    # Parse Building Type
    building_type = parse_building_type(title)
    
    # Parse Full CAR Type
    full_car_type = parse_full_car_type(title)
    
    # Pasre if title contains "Bulk"
    bulk_string = 'Bulk ' if 'bulk' in title.lower() else ''
    
    # Parse Region
    region = parse_region(title, template_id)
            
    return({'Region':region, 
            'POR Year':por_year, 
            'CAR Type':car_type, 
            'Current Request ($ MM)':current_request, 
            'Unrecognized':unrecongized, 
            'Building Type':building_type, 
            'Site':site,
            'Full CAR Type':full_car_type,
            'Clean Title': bulk_string+site+' '+building_type+' '+full_car_type})

def extract_info(df, template_name):
    if template_name is None:
        df_template = df
    else:
        df_template = df[df['Template Id'] == template_dict[template_name]].reset_index(drop=True)
    
    df_nbd = get_nbd(df_template)
    df_actors = pivot_actor_action(df_template)
    
    df_pivot = df_nbd.merge(df_actors, on=['Template Id','Approval Id','Title'], how='outer')
    
    # Complete all actor actions. If there is any missing action, create a null column for it.
    for i in ['Pending', 'Pending in future', 'Approved', 'Approved with condition']:
        if i not in df_pivot.columns:
            df_pivot[i] = ''
    
    df_approver_check_list = get_approver_check_list(df_pivot)
    
    df_info = df_pivot.join(df_approver_check_list)
    
#     both_vp_is_na = (df_info['ABD'] == DEFAULT_MARK) & (df_info['OG'] == DEFAULT_MARK)
#     both_vp_is_approved = (df_info['ABD'] == CHECK_MARK) & (df_info['OG'] == CHECK_MARK)
#     df_info = df_info[(~both_vp_is_na) & (~both_vp_is_approved)].reset_index(drop=True)

    
    vp_pending = (df_info['SP'] == PENDING_MARK) | (df_info['DH'] == PENDING_MARK) | (df_info['MN'] == PENDING_MARK) | (df_info['RW'] == PENDING_MARK)
    df_info = df_info[vp_pending].reset_index(drop=True)
    
    df_info = determine_queue(df_info)
    
    df_title_info = pd.DataFrame(list(df_info.apply(lambda x: extract_info_from_title(x['Title'], x['Template Id']), axis=1)))
    
    df_info = df_info.join(df_title_info)
    
    df_info['SORT_ORDER_REGION'] = [SORT_ORDER_REGION.get(str(c)) if SORT_ORDER_REGION.get(str(c)) is not None else 9999 for c in df_info['Region']]
    
    require_field = ['Queue','Region','POR Year','CAR Type','Clean Title','Current Request ($ MM)',
#                      'Approval Needed By','Approval Id','Template Id','ABD','OG','MM','MN','TR','RW',
                     'Approval Needed By','Approval Id','Template Id','SP','DH','MN','RW',
                     'Pending', 'Pending in future', 'Approved', 'Approved with condition',
                     'Site','Building Type','Full CAR Type','Title']
    
    df_output = df_info.sort_values(['Queue','CAR Type','SORT_ORDER_REGION','POR Year'], ascending=[False,True,True,True]).reset_index(drop=True)
    df_output = df_output[require_field]
    df_output.reset_index(inplace=True)

    df_output['index'] = df_output['index'] + 1
    df_output['Title'] = df_output['Title'].apply(preprocess_title)
    df_output['Approval Id'] = df_output['Approval Id'].apply(add_hyper_link_to_approval)
    df_output['Approval Needed By'] = format_nbd(df_output['Approval Needed By'])
    
    df_output.rename(columns={'index':'#',
                              'Approval Needed By':'Need By Date',
                              'Title':'Original Title', 
                              'Clean Title':'Title',
                              'Pending':'CAR Approvers Update'}, inplace=True)
    
    return(df_output)

In [9]:
def highlight_checked_list(s):
    
    style_list = []
    
    for v in s:
        if v == CHECK_MARK:
            style = 'background-color: green; text-align: center'
        elif v == DEFAULT_MARK:
            style = 'background-color: gray; text-align: center'
        else:
            style = ''
        
        style_list.append(style)
    
    return(style_list)

def format_to_one_digit_decimal(s):
    return(['float-format: %0.1f'] * len(s))

def highlight_past_due_nbd(s):
#     nbd = pd.to_datetime(([str(dt.now().year)+'-'] + s), format='%Y-%d-%b')
    nbd = pd.to_datetime(s, format='%d-%b, %Y')
    v = dt.now() > nbd
    
    return(['background-color: #FFEEBB; text-align: right' if i else 'text-align: right' for i in v])

def right_align_por_year(s):
    return(['text-align: right'] * len(s))

In [10]:
df_all = extract_info(df=df, template_name=None)
df_all.head()

   Template Id  Approval Id  \
0            0      8095089   
1            0      8221312   
2            0      8321663   
3            0      8523194   
4            0      9180485   

                                               Title   Approval Needed By  
0  NA | WWRE CAR Approval | 2022 | SPP | Land Acq...  08-30-2022 06:59:59  
1  NA | NACF CAR Approval | 2022 | Reverse Logist...  08-26-2022 06:59:59  
2  EUCF CAR Approval: 2022 AR &ndash; LCY2 (Tilbu...  11-14-2022 23:59:59  
3  NA | NACF CAR Approval | 2022 | IXD | Construc...  08-26-2022 06:59:59  
4  INCF CAR approval: 2023 Sort centre &ndash; MD...  06-20-2022 18:29:59  
pd.DataFrame(output)
     SP  DH  MN  RW
0    NA      NA  NA
1    NA  NA  NA    
2    NA  NA  NA  NA
3                  
4    NA  NA  NA  NA
..   ..  ..  ..  ..
162  NA  NA  NA  NA
163  NA            
164  NA  NA  NA  NA
165  NA  NA  NA  NA
166  NA  NA  NA  NA

[167 rows x 4 columns]
    Template Id  Approval Id  \
0             0      8095089   
1       

,#,Queue,Region,POR Year,CAR Type,Title,Current Request ($ MM),Need By Date,Approval Id,Template Id,...,MN,RW,CAR Approvers Update,Pending in future,Approved,Approved with condition,Site,Building Type,Full CAR Type,Original Title
0,1,VP,NA,2022,Initiative,,NaN,"12-Aug, 2022","=HYPERLINK(""https://approval.amazon.com/Approv...",0,...,,,"Nick, Melissa | Williams (Finance), Russel","Capital-Planning | Heimburger, Doug","Budge, Richard | Odisho, Walt | Peltz, Emily |...",,,,,NA | NACF Initiative CAR Approval | 2022 | Tra...
1,2,VP,EU,2024,RE,STN6 ARS,730.6,"25-Nov, 2022","=HYPERLINK(""https://approval.amazon.com/Approv...",0,...,NA,NA,"Heimburger, Doug","Capital-Planning | Carpenter, Gail | Herringto...","Argentieri, Andrea | Gal, Jonatan | EU-Capital...",,STN6,ARS,,EUCF CAR Approval: 2024 BTS ARS 550 G+3 547k (...
2,3,VP,EU,2024,RE,BVA3,173.2,"01-Dec, 2022","=HYPERLINK(""https://approval.amazon.com/Approv...",0,...,NA,NA,"Heimburger, Doug","Capital-Planning | Felton (Seattle), John | po...","Lewkowitz, David | Mallory, Daniel | Kalogerop...",,BVA3,,,"EUCF CAR Approval: 2024 BAR - BVA3 (Beauvais, ..."
3,4,VP,NA,2024,RE,SAN6 Construction,34.4,"09-Dec, 2022","=HYPERLINK(""https://approval.amazon.com/Approv...",0,...,✓,✓,"Heimburger, Doug","Capital-Planning | Felton (Seattle), John","Shankar, Reshma | Westwood, Kaitlyn | Perego, ...",,SAN6,,Construction,NA | NACF CAR Approval | 2024 | Traditional No...
4,5,VP,NA,2023,SU,CLE7 Construction,18.3,"28-Nov, 2022","=HYPERLINK(""https://approval.amazon.com/Approv...",0,...,✓,✓,"Heimburger, Doug",Capital-Planning,"Shankar, Reshma | Westwood, Kaitlyn | LeMasuri...",,CLE7,,Construction,NA | NACF CAR Approval | 2023 | Print on Dema...


In [11]:
new_set = set(df_all['Template Id'])
old_set = set([39820, 39818, 39802, 37508, 37158, 37156, 37155, 36470, 35476, 35306, 30143, 7394, 7393])
new_set - old_set

{0}

In [12]:
required_field = ['#','Queue','Region','POR Year','CAR Type','Title','Current Request ($ MM)','Need By Date','Approval Id',
#                    'ABD','OG','MM','MN','TR','RW','CAR Approvers Update','Original Title']
                   'SP','DH','MN','RW','CAR Approvers Update','Original Title']

df_output = df_all[required_field]
# styled = df_output.style.apply(highlight_checked_list, subset=['ABD','OG','MM','MN','TR','RW']).\
styled = df_output.style.apply(highlight_checked_list, subset=['SP','DH','MN','RW']).\
                        set_properties(subset=['Approval Id'], **{'color': 'blue'}).\
                        apply(format_to_one_digit_decimal,subset=['Current Request ($ MM)']).\
                        apply(highlight_past_due_nbd, subset=['Need By Date']).\
                        apply(right_align_por_year, subset=['POR Year'])

styled

,#,Queue,Region,POR Year,CAR Type,Title,Current Request ($ MM),Need By Date,Approval Id,SP,DH,MN,RW,CAR Approvers Update,Original Title
0,1,VP,NA,2022,Initiative,,nan,"12-Aug, 2022","=HYPERLINK(""https://approval.amazon.com/Approval/Details/9664741"", 9664741)",NA,,,,"Nick, Melissa | Williams (Finance), Russel","NA | NACF Initiative CAR Approval | 2022 | Traditional Non-Sortable | DeStuff-It/Cascading Belt Conveyance Floor Load Solve | Multi Site | Final Full | Initiative | 16,963,474 USD"
1,2,VP,EU,2024,RE,STN6 ARS,730.600000,"25-Nov, 2022","=HYPERLINK(""https://approval.amazon.com/Approval/Details/11519227"", 11519227)",✓,,NA,NA,"Heimburger, Doug","EUCF CAR Approval: 2024 BTS ARS 550 G+3 547k (Gen 10) - STN6 (Northampton, UK) Real Estate - ($730.6MM)"
2,3,VP,EU,2024,RE,BVA3,173.200000,"01-Dec, 2022","=HYPERLINK(""https://approval.amazon.com/Approval/Details/11552473"", 11552473)",✓,,NA,NA,"Heimburger, Doug","EUCF CAR Approval: 2024 BAR - BVA3 (Beauvais, France) Real Estate RE - ($173,190,947)"
3,4,VP,NA,2024,RE,SAN6 Construction,34.400000,"09-Dec, 2022","=HYPERLINK(""https://approval.amazon.com/Approval/Details/11856821"", 11856821)",✓,,✓,✓,"Heimburger, Doug","NA | NACF CAR Approval | 2024 | Traditional Non-Sortable | Established | SAN6 | San Diego, CA, United States | Supplemental | Construction | $34.4MM (Project Total: $301.8MM)"
4,5,VP,NA,2023,SU,CLE7 Construction,18.300000,"28-Nov, 2022","=HYPERLINK(""https://approval.amazon.com/Approval/Details/12133418"", 12133418)",NA,,✓,✓,"Heimburger, Doug","NA | NACF CAR Approval | 2023 | Print on Demand | Co-location | CLE7 | North Randall, OH, United States | Initial Full | Construction and Startup | $18.3MM CN, $47.3MM SU (Aggregate Total $65.6MM)"
5,6,VP,NA,2024,SU,,610.100000,"14-Oct, 2022","=HYPERLINK(""https://approval.amazon.com/Approval/Details/11584089"", 11584089)",✓,,✓,✓,"Heimburger, Doug","NA | NACF CAR Approval | 2024 | AR Sortable | AR and MHE Pull Forward | Multi Site | Multi, Multi, United States | Long Lead | Startup | Aggregate Total $610.1MM USD"
6,7,Pre-VP,AP,2023,Initiative,BWU2,5.400000,"02-Dec, 2022","=HYPERLINK(""https://approval.amazon.com/Approval/Details/13224295"", 13224295)",NA,,NA,NA,"Arellano, Domingo | Gieraltowski, Bart | Espidio-Garcia, Mindy | Bharadwaj, Ankit","APAC | EC CF CAR Approval | 2023 | AR Sortable | Sustainability | BWU2 | Kemps Creek, NSW, Australia | Final Full | Initiative | $5.4MM USD"
7,8,Pre-VP,NA,2022,Initiative,,0.000000,"13-Sep, 2022","=HYPERLINK(""https://approval.amazon.com/Approval/Details/11030388"", 11030388)",NA,,,✓,CARApprovalHold,"NA | NACF CAR Approval | 2022 | AR Sortable | AMCARE to Wellness Center | Multi Site | Multi, Multi, United States | Supplemental | Initiative | $1.52 USD"
8,9,Pre-VP,NA,2022,Initiative,FAT1,3.600000,"11-Oct, 2022","=HYPERLINK(""https://approval.amazon.com/Approval/Details/11729569"", 11729569)",NA,NA,NA,,na-finance,"NA | NACF CAR Approval | 2022 | AR Sortable | Secure Parking Lot | FAT1 | Fresno, CA, United States | Final Full | Initiative | $3.6MM USD"
9,10,Pre-VP,NA,2022,Initiative,SDF9,0.500000,"31-Oct, 2022","=HYPERLINK(""https://approval.amazon.com/Approval/Details/12054232"", 12054232)",NA,NA,NA,,"Haywood, Richard","NA | NACF Initiative CAR Approval | 2022 | Reverse Logistics| UNSELLABLE CAPACITY ALIGNMENT | SDF9 | Shepherdsville, KY | Final Full | Initiative | $533,313 USD"


In [13]:
styled.to_excel(new_approval_report_output_file_path, engine='openpyxl', index=False)

# Combine with Previous Commented File

In [14]:
import openpyxl
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Border, Side, Font, Alignment
from openpyxl.styles import DEFAULT_FONT

from copy import copy

In [15]:
wb_last_edit = load_workbook(last_edit_report)
ws_last_edit = wb_last_edit[wb_last_edit.sheetnames[0]]

wb_new_report = load_workbook(new_report)
ws_new_report = wb_new_report[wb_new_report.sheetnames[0]]

In [16]:
legend_dict = {'D2': {'text':'Approved',                      'hex':'00C0C0C0'}, 
               'D3': {'text':'Cancelled',                     'hex':'00FF0000'},
               'D4': {'text':'Past Need By Date',             'hex':'00FFEEBB'},
               'E2': {'text':'Check for VP Queue Next Email', 'hex':'0000CCFF'},
               'E3': {'text':'Offline',                       'hex':'00FFCC00'},
               'E4': {'text':'Hold',                          'hex':'00FF9900'},
               'F2': {'text':'VP Feedback Provided',          'hex':'00FFFF00'},
               'F3': {'text':'Reponse Provided',              'hex':'0099CC00'}}

thin_border = Border(left=Side(style='thin'), 
                     right=Side(style='thin'), 
                     top=Side(style='thin'), 
                     bottom=Side(style='thin'))

hyper_link_font = Font(name='Calibri',
                       size=11,
                       bold=False,
                       italic=False,
                       vertAlign=None,
                       underline='single',
                       strike=False,
                       color='0070c0')

header_row_font = Font(name='Calibri',
                       size=11,
                       bold=True,
                       italic=False,
                       vertAlign=None,
                       underline='none',
                       strike=False,
                       color='FF000000')

header_row_fill_hex = '00C0C0C0'

In [17]:
excel_offset = 1 # row & column start from 1 instead of 0

start_row_last_edit = 7
start_row_new_report = 2

end_row_last_edit = len(ws_last_edit['A'])
end_row_new_report = len(ws_new_report['A'])

cells_last_edit = ws_last_edit['B{start_row}:I{end_row}'.format(start_row=start_row_last_edit, 
                                                                end_row=end_row_last_edit)]
cells_new_report = ws_new_report['B{start_row}:I{end_row}'.format(start_row=start_row_new_report, 
                                                                  end_row=end_row_new_report)]

# Convert Need by Date from string to datetime
for rows in ws_new_report['H2:H{end_row}'.format(end_row=end_row_new_report)]:
    for cell in rows:
#         cell.value = dt.strptime(str(dt.now().year)+'-'+cell.value, '%Y-%d-%b')
        cell.value = dt.strptime(cell.value, '%d-%b, %Y')

# Add New Approval Validation Column
new_approval_col = 16
new_approval_title = ws_new_report.cell(row=1, column=new_approval_col)
new_approval_title.value = 'New Entity'

for i in range(2, end_row_new_report+1):
    cell_to_write = ws_new_report.cell(row=i, column=new_approval_col)
    cell_to_write.value = True

# Add Is New Request Amount Column to determine if the request amount changed from previous report
is_new_request_amount_col = 17
is_new_request_amount_title = ws_new_report.cell(row=1, column=is_new_request_amount_col)
is_new_request_amount_title.value = 'Is New Request Amount'

for i in range(2, end_row_new_report+1):
    cell_to_write = ws_new_report.cell(row=i, column=is_new_request_amount_col)
    cell_to_write.value = 'No'

# Initiate New Request Amount
request_amount_col = 7
new_request_amount_col = 18
new_request_amount_title = ws_new_report.cell(row=1, column=new_request_amount_col)
new_request_amount_title.value = 'New Request Amount'

for i in range(2, end_row_new_report+1):
    cell_to_write = ws_new_report.cell(row=i, column=new_request_amount_col)
    cell_to_write.value = ws_new_report.cell(row=i, column=request_amount_col).value

In [18]:
df_last_edit = pd.DataFrame(ws_last_edit.values)

column_approval_id_ws_last_edit = 8
column_approval_id_cells_last_edit = len(cells_last_edit[0]) - 1
approval_ids_last_edit = [int(str(i).split(', ')[1].replace(')','')) if 'HYPERLINK' in str(i) else 0 for i in list(df_last_edit[column_approval_id_ws_last_edit])]
print(approval_ids_last_edit)

for r in range(len(cells_new_report)):
    approval_id = int(cells_new_report[r][column_approval_id_cells_last_edit].value.split(', ')[1].replace(')',''))
    
    if approval_id in approval_ids_last_edit:
        r_last_edit = approval_ids_last_edit.index(approval_id) + excel_offset - start_row_last_edit
        
        # Mark if an entity is not an new entity
        ws_new_report.cell(row=r+2, column=new_approval_col).value = False

        # Mark if an entity's request amount change
        request_amount_col = 7
        new_request_amount = cells_new_report[r][request_amount_col-2].value
        last_edit_request_amount = cells_last_edit[r_last_edit][request_amount_col-2].value
        
        try:
            if (round(float(new_request_amount),1) != round(float(last_edit_request_amount),1)):
                ws_new_report.cell(row=r+2, column=is_new_request_amount_col).value = 'Yes'
            
            ws_new_report.cell(row=r+2, column=new_request_amount_col).value = new_request_amount
        
        except:
            print('last_edit_request_amount: ', last_edit_request_amount)
            print('new_request_amount: ', new_request_amount)
        
        for c in range(len(cells_new_report[r]) - 1):
            cell = cells_last_edit[r_last_edit][c]
            new_cell = cells_new_report[r][c]
            
            if new_cell.column_letter not in ['B','H']:
                new_cell.value = cell.value

            if cell.has_style and cell.column_letter not in ['G','H']:
                new_cell.border = copy(cell.border)
                new_cell.number_format = copy(cell.number_format)
                new_cell.protection = copy(cell.protection)
                new_cell.alignment = copy(cell.alignment)
                new_cell.font = copy(cell.font)

                if cell.fill.fgColor.type == 'indexed' and cell.fill.fgColor.indexed==10 and cell.fill.bgColor.rgb=='00000000':
                    pass
                else:
                    new_cell.fill = copy(cell.fill)

[0, 0, 0, 0, 0, 0, 12983059, 9664741, 11519227, 11552473, 8562544, 11856821, 12133418, 11584089, 12614911, 13224295, 11729569, 12054232, 12174279, 12256162, 12276004, 12350216, 12394042, 12606044, 12857414, 13101206, 13276116, 11744526, 12250389, 12414013, 12415416, 13196261, 13222826, 13228106, 13280065, 12372386, 12927174, 13018954, 11838641, 12738574, 13127869, 13316423, 13253522, 12267076, 12890632, 13247214, 10934694, 10934834, 12953282, 13195489, 13199163, 13199373, 10496519, 12162308, 12849404, 12854562, 12936880, 11388370, 12025073, 12273397, 12621841, 11030388, 12131068, 12618541, 8095089, 8221312, 9967729, 10521969, 11352459, 11896160, 12163973, 8523194, 11350398, 11351684, 11352232, 11560990, 9276487, 11598461]
last_edit_request_amount:  17
new_request_amount:  None


In [19]:
# Format Current Request
for rows in ws_new_report['G{start_row}:G{end_row}'.format(start_row=start_row_new_report, end_row=end_row_new_report)]:
    for cell in rows:
        cell.number_format = '#,##0.0'
    
# Format Font for whole sheet
for rows in ws_new_report['A1:Z{end_row}'.format(end_row=end_row_new_report)]:
    for cell in rows:
        cell.font = DEFAULT_FONT

# Format Border for whole sheet
for rows in ws_new_report['A1:F{end_row}'.format(end_row=end_row_new_report)]:
    for cell in rows:
        cell.border = thin_border  
for rows in ws_new_report['P1:R{end_row}'.format(end_row=end_row_new_report)]:
    for cell in rows:
        cell.border = thin_border

# Format Need By Date
for cell in ws_new_report['H']:
    cell.number_format = 'd-mmm, Y'
# for row in ws_new_report['H7:H{end_row}'.format(end_row=end_row_new_report)]:
#     for cell in rows:
#         cell.number_format = 'd-mmm'

# Format Approval Id URL
for rows in ws_new_report['I{start_row}:I{end_row}'.format(start_row=start_row_new_report, end_row=end_row_new_report)]:
    for cell in rows:
        cell.font = hyper_link_font

# Format Title Cell Width
ws_new_report.column_dimensions['A'].width = 5
ws_new_report.column_dimensions['F'].width = 50
ws_new_report.column_dimensions['H'].width = 12
ws_new_report.column_dimensions['N'].width = 25
ws_new_report.column_dimensions['O'].width = 30
for c in ['J','K','L','M']:
    ws_new_report.column_dimensions[c].width = 6

# Format Header row
for rows in ws_new_report['A1:N1']:
    for cell in rows:
        cell.font = header_row_font
        cell.fill = PatternFill(start_color=header_row_fill_hex, 
                                end_color=header_row_fill_hex, 
                                fill_type = "solid")

# Format all sheet cell with vertical alignment
for rows in ws_new_report['A{start_row}:I{end_row}'.format(start_row=start_row_new_report, end_row=end_row_new_report)]:
    for cell in rows:
        cell.alignment = Alignment(vertical='top')
for rows in ws_new_report['J{start_row}:M{end_row}'.format(start_row=start_row_new_report, end_row=end_row_new_report)]:
    for cell in rows:
        cell.alignment = Alignment(vertical='top', horizontal='center')
for rows in ws_new_report['N{start_row}:R{end_row}'.format(start_row=start_row_new_report, end_row=end_row_new_report)]:
    for cell in rows:
        cell.alignment = Alignment(vertical='top')

# Align POR Year column to left
for rows in ws_new_report['D{start_row}:D{end_row}'.format(start_row=start_row_new_report, end_row=end_row_new_report)]:
    for cell in rows:
        cell.alignment = Alignment(vertical='top', horizontal='left')

In [20]:
# Add Filter
ws_new_report.auto_filter.ref = 'B6:R{end_row}'.format(end_row=len(ws_new_report['A']))
ws_new_report.auto_filter.idx_base = 6

In [21]:
ws_new_report.insert_rows(1,5)

for k, v in legend_dict.items():
    ws_new_report[k].value = v['text']
    ws_new_report[k].fill = PatternFill(start_color=v['hex'], end_color=v['hex'], fill_type = "solid")
    
ws_new_report.row_dimensions[6].height = 40

In [22]:
wb_new_report.save(final_output_file_path)